# Deep Learning with PyTorch Step-by-Step: A Beginner's Guide

# Data Loader

In [1]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, TensorDataset, DataLoader

## Helper Function

In [2]:
def make_train_step(model, loss_fn, optimizer):
    # Builds function that performs a step in the train loop
    def perform_train_step(x, y):
        # Sets model to TRAIN mode
        model.train()
        
        # Step 1 - computes model's predictions - forward pass
        yhat = model(x)
        # Step 2 - computes the loss
        loss = loss_fn(yhat, y)
        # Step 3 - computes gradients for "b" and "w" parameters
        loss.backward()
        # Step 4 - updates parameters using gradients and
        # the learning rate
        optimizer.step()
        optimizer.zero_grad()
        
        # Returns the loss
        return loss.item()
    
    # Returns the function that will be called inside the 
    # train loop
    return perform_train_step

In [11]:
# This is redundant now, but it won't be when we introduce 
# Datasets...
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Sets learning rate - this is "eta" ~ the "n" like Greek letter
lr = 0.1

torch.manual_seed(42)
# Now we can create a model and send it at once to the device
model = nn.Sequential(nn.Linear(1, 1)).to(device)

# Defines a SGD optimizer to update the parameters 
# (now retrieved directly from the model)
optimizer = optim.SGD(model.parameters(), lr=lr)

# Defines a MSE loss function
loss_fn = nn.MSELoss(reduction='mean')

## Data Generation

In [12]:
true_b = 1
true_w = 2
N = 100
train_step = make_train_step(model, loss_fn, optimizer)

# Data Generation
np.random.seed(42)
x = np.random.rand(N, 1)
epsilon = (.1 * np.random.randn(N, 1))
y = true_b + true_w * x + epsilon

### Generating training and validation sets

In [4]:
# Shuffles the indices
idx = np.arange(N)
np.random.shuffle(idx)

# Uses first 80 random indices for train
train_idx = idx[:int(N*.8)]
# Uses the remaining indices for validation
val_idx = idx[int(N*.8):]

# Generates train and validation sets
x_train, y_train = x[train_idx], y[train_idx]
x_val, y_val = x[val_idx], y[val_idx]

## Full Pipeline

### Data Preparation

In [6]:
# %%writefile data_preparation/v1.py

# Our data was in Numpy arrays, but we need to transform them
# into PyTorch's Tensors
x_train_tensor = torch.as_tensor(x_train).float()
y_train_tensor = torch.as_tensor(y_train).float()

# Builds Dataset
train_data = TensorDataset(x_train_tensor, y_train_tensor)  

# Builds DataLoader
train_loader = DataLoader(                                  
    dataset=train_data, 
    batch_size=16, 
    shuffle=True,
)

In [7]:
train_loader

In [ ]:
%run -i data_preparation/v1.py

### Model Configuration

In [ ]:
%run -i model_configuration/v1.py

### Model Training

In [13]:
# %%writefile model_training/v2.py

# Defines number of epochs
n_epochs = 100

losses = []

# For each epoch...
for epoch in range(n_epochs):
    # inner loop
    mini_batch_losses = []                              
    for x_batch, y_batch in train_loader:               
        # the dataset "lives" in the CPU, so do our mini-batches
        # therefore, we need to send those mini-batches to the
        # device where the model "lives"
        x_batch = x_batch.to(device)                    
        y_batch = y_batch.to(device)                    

        # Performs one train step and returns the 
        # corresponding loss for this mini-batch
        mini_batch_loss = train_step(x_batch, y_batch)  
        mini_batch_losses.append(mini_batch_loss)       

    # Computes average loss over all mini-batches
    # That's the epoch loss
    loss = np.mean(mini_batch_losses)                       
    losses.append(loss)

In [14]:
losses

[np.float64(0.3474797248840332),
 np.float64(0.0818866990506649),
 np.float64(0.06159851998090744),
 np.float64(0.05324689298868179),
 np.float64(0.047001594305038454),
 np.float64(0.042160157859325406),
 np.float64(0.03744911104440689),
 np.float64(0.032614246010780334),
 np.float64(0.02898256704211235),
 np.float64(0.0266994196921587),
 np.float64(0.023629816621541976),
 np.float64(0.021306158415973185),
 np.float64(0.019503521732985973),
 np.float64(0.017948344722390173),
 np.float64(0.016558653302490713),
 np.float64(0.015389558672904969),
 np.float64(0.014326035231351852),
 np.float64(0.013402503728866578),
 np.float64(0.01279139444231987),
 np.float64(0.011995002999901772),
 np.float64(0.011511744186282157),
 np.float64(0.010993200074881315),
 np.float64(0.010582373477518559),
 np.float64(0.0102280518040061),
 np.float64(0.009885442815721035),
 np.float64(0.009756661485880613),
 np.float64(0.009399115107953548),
 np.float64(0.009250048734247684),
 np.float64(0.00910855121910572),

In [ ]:
%run -i model_training/v2.py

## Model Parameters

In [ ]:
# printing the parameter values of the Linear model
print(model.state_dict())